In [1]:
import nltk
import pandas as pd
import re
import time
import multiprocessing
import io
# import gensim
import numpy as np
import matplotlib.pyplot as plt
import pickle
# import keras
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
# from tensorflow import keras
# from keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint
# from keras.preprocessing.text import Tokenizer
# from keras.models import Sequential
# from keras.layers import Embedding
# from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
# from keras import regularizers
# from tensorflow.keras.utils import to_categorical
# from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import vektor_tfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
# from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#2.Load Data

In [54]:
#simpan path dataset
path_data_sentimen = "data_scraping_kota_TGL.csv"
data_sentimen = pd.read_csv(path_data_sentimen, sep=",", header=[0], encoding="UTF-8")

In [55]:
data_sentimen.columns = ['Unnamed: 0', 'date', 'username', 'tweet', 'wisata_hiburan', 'pendidikan', 'fasilitas_layanan_publik', 'kuliner']
data_sentimen.head()

,Unnamed: 0,date,username,tweet,wisata_hiburan,pendidikan,fasilitas_layanan_publik,kuliner
0,0,09/12/2021 23:11,BsiWahyudi,Wisuda Universitas BSI Kampus Kabupaten Banyum...,-1,1,-1,-1
1,1,09/12/2021 20:44,PPPLarangan,(03/12/2021) Mengikuti kegiatan bersih pantai...,-1,-1,1,-1
2,2,09/12/2021 19:46,AdvFeryAjah,"@infoplntegal Selamat malam , mohon di tindak ...",-1,-1,0,-1
3,3,09/12/2021 19:27,tegalfess,"[TF] info banjir daerah tegal kota dong lurr, ...",-1,-1,-1,1
4,4,09/12/2021 14:58,detikcom,Pemasangan portal di jalan masuk Alun-alun Kot...,-1,-1,0,-1


#3.Implementasi Preprocessing

In [5]:
#menyimpan tweet. (tipe data series pandas)
data_content = data_sentimen['tweet']

##3.1. Casefolding

In [6]:
# casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    wisuda universitas bsi kampus kabupaten banyum...
1    (03/12/2021)  mengikuti kegiatan bersih pantai...
2    @infoplntegal selamat malam , mohon di tindak ...
3    [tf] info banjir daerah tegal kota dong lurr, ...
4    pemasangan portal di jalan masuk alun-alun kot...
Name: tweet, dtype: object

##3.2. Filtering

In [7]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


In [8]:
data_filtering

0      wisuda universitas bsi kampus kabupaten banyum...
1               mengikuti kegiatan bersih pantai yang...
2       infoplntegal selamat malam   mohon di tindak ...
3       tf  info banjir daerah tegal kota dong lurr  ...
4      pemasangan portal di jalan masuk alun alun kot...
                             ...                        
375    sekda  kota tegal johardi mempersilakan jika a...
376     tegalfess mie ayam solo baru ini yg tegal kot...
377    senin        ratih  m  luhkan kota tegal  di m...
378     tf  saran sate taichan di tegal kota yg enak ...
379     tf  gaes  rekomendasiin mie ayam enak di tega...
Length: 380, dtype: object

## 3.3. Tokenisasi

In [9]:
# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

In [10]:
data_tokenize[:5]

[['wisuda',
  'universitas',
  'bsi',
  'kampus',
  'kabupaten',
  'banyumas',
  'ke',
  'amp',
  'kampus',
  'kota',
  'tegal',
  'ke',
  'java',
  'heritage',
  'hotel',
  'purwokerto'],
 ['mengikuti',
  'kegiatan',
  'bersih',
  'pantai',
  'yang',
  'diselenggarakan',
  'lanal',
  'tegal',
  'di',
  'pantai',
  'alam',
  'indah',
  'kota',
  'tegal',
  'kegiatan',
  'ini',
  'diikuti',
  'oleh',
  'berbagai',
  'instansi',
  'terkait',
  'di',
  'kota',
  'tegal',
  'dan',
  'sekitarnya',
  'dkpjateng',
  'djpt',
  'kkp'],
 ['infoplntegal',
  'selamat',
  'malam',
  'mohon',
  'di',
  'tindak',
  'lanjuti',
  'di',
  'daerah',
  'saya',
  'dari',
  'jam',
  'wib',
  'mati',
  'listrik',
  'sampai',
  'sekarang',
  'belum',
  'menyala',
  'alamat',
  'di',
  'jl',
  'drcipto',
  'mangunkusumo',
  'kaligansa',
  'kecamatan',
  'margadana',
  'kota',
  'tegal'],
 ['tf',
  'info',
  'banjir',
  'daerah',
  'tegal',
  'kota',
  'dong',
  'lurr',
  'kayong',
  'pgn',
  'seblak',
  'kie',

##3.4. Konversi Slangword

In [ ]:
#slang word
path_dataslang = open("kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

## 3.5. Stopword

In [12]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
reviews = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\novit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# pickle.dump(reviews, open("/content/drive/My Drive/Thesis/Tesis/program/data/model/NVB/reviews.pickle", "wb"))

# 4.Implementasi Naive Bayes (Klasifikasi Aspek)



## 4.1.pembuatan vector kata

In [24]:
# pembuatan vector kata
# reviews = pickle.load(open("/content/drive/My Drive/Thesis/Tesis/program/data/model/NVB/reviews.pickle", "rb"))
vectorizer = TfidfVectorizer()
reviews2 = [" ".join(r) for r in reviews]
vektor_tfidf = vectorizer.fit_transform(reviews2)

## Aspek Wisata

### Modeling

In [56]:
class READ_DATA:
      aspek = ''
      vektor_tfidf = ''
      data_sentimen_wisata_hiburan = ""
      data_sentimen_pendidikan = "" 
      data_sentimen_fasilitas_layanan_publik = ""
      data_sentimen_kuliner = "" 
      # data_sentimen_kontinuitas = ""
      def __init__(self, data_sentimen, vektor_tfidf):
          self.data_sentimen_wisata_hiburan = data_sentimen['wisata_hiburan']
          self.data_sentimen_pendidikan = data_sentimen['pendidikan']
          self.data_sentimen_fasilitas_layanan_publik = data_sentimen['fasilitas_layanan_publik']
          self.data_sentimen_kuliner = data_sentimen['kuliner']
          self.vektor_tfidf = vektor_tfidf

      def get_data_sentimen(self, aspek):
          data_sentimen= getattr(self, f'data_sentimen_{aspek}')
          data_sentimen_list = data_sentimen.to_list()
          # print(type(data_sentimen_list[0]))

          #filter hapus data yang tidak memiliki sentimen (-1)
          x_, y_ = [],[]    
          for i, j in zip(vektor_tfidf, data_sentimen_list):
              if j != -1:
                  x_.append(i)
                  y_.append(j)
          return x_, y_

In [73]:
# Ubah kategori
read_data = READ_DATA(data_sentimen, vektor_tfidf)
x_content_sentimen_NVB, y_label_sentimen_NVB = read_data.get_data_sentimen('wisata_hiburan')

In [74]:
from sklearn.utils.multiclass import unique_labels
unique_labels(y_label_sentimen_NVB)

array([0, 1])

In [77]:
X_train, X_test, y_train, y_test = train_test_split(x_content_sentimen_NVB, y_label_sentimen_NVB, test_size=0.2)#, shuffle=True)
# nama = "Model_Sentimen_Wisata_Hiburan"
# y_train = np.array(y_train)
# y_test = np.array(y_test)

In [78]:
from imblearn.over_sampling import ADASYN

X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)

ValueError: Expected 2D array, got 1D array instead:
array=[<1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 10 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 6 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 26 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 3 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 8 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 13 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 6 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 8 stored elements in Compressed Sparse Row format>
 <1x2023 sparse ma

In [79]:
from sklearn import metrics
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
print("Akurasi : "+str(clf.score(X_train, y_train)))

print("Precission : "+str(metrics.precision_score(y_train, y_pred, labels=[0,1], average='macro')))

ValueError: Expected 2D array, got 1D array instead:
array=[<1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 10 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 6 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 26 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 3 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 8 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 13 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 6 stored elements in Compressed Sparse Row format>
 <1x2023 sparse matrix of type '<class 'numpy.float64'>'
 	with 8 stored elements in Compressed Sparse Row format>
 <1x2023 sparse ma

In [ ]:
# save model
# model_nvb = 'vektor_tfidf_sentimen_integritas_nvb.sav'
pickle.dump(clf, open('model/vektor_tfidf_{}_nvb.pkl'.format(nama), 'wb'))

In [ ]:
model = pickle.load(open('model/vektor_tfidf_'+str(nama)+'_nvb.pkl','rb'))
pred_y_ = model.predict(X_test)
label_test = y_test
# result = model.score(X_test, y_test)
print('\n'+'-'*45)
print('confusion matrix : ' + str(nama))
print('-'*45,'\n')
print(classification_report(label_test, pred_y_, labels=[-1, 0, 1]))


---------------------------------------------
confusion matrix : Model_Aspek_Wisata_Hiburan
--------------------------------------------- 

              precision    recall  f1-score   support

          -1       1.00      0.86      0.93        95
           0       0.12      1.00      0.22         1
           1       0.67      0.89      0.76        18

    accuracy                           0.87       114
   macro avg       0.60      0.92      0.64       114
weighted avg       0.94      0.87      0.89       114



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vektor_vektor_tfidf, sentimen_kontinuitas, test_size=0.2)
nama_sentimen = nameof(sentimen_kontinuitas)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# Cross-validasi
kf = StratifiedKFold(n_splits=2)#,shuffle=True, random_state=None) # Define the split - into 5 folds 
kf.get_n_splits(X_train) # returns the number of splitting iterations in the cross-validator
print(kf)

StratifiedKFold(n_splits=2, random_state=None, shuffle=False)


# 5.Klasifikasi

In [ ]:
def hitung_score_multiclass(y_test_sentimen, y_pred):
    # cm = metrics.confusion_matrix(y_test_sentimen, y_pred, labels=[-1,0,-1])
    mcm = multilabel_confusion_matrix(y_test_sentimen, y_pred, labels=[-1,0,1])
    # print(cm)
    print('\n'+'-'*50)
    print('confusion matrix : ' + str(nama_sentimen))
    print('-'*50)
    print(plot(label_asli,pred_y_nvb),'\n')
    tn = mcm[:, 0, 0]
    tp = mcm[:, 1, 1]
    fn = mcm[:, 1, 0]
    fp = mcm[:, 0, 1]
    print("TP : " + str(tp),
          "TN : " +str(tn),
          "FP : " +str(fp),
          "FN : " +str(fn),'\n')
    acc = accuracy_score(y_test_sentimen, y_pred)
    prec_mi = precision_score(y_test_sentimen, y_pred, average='micro')
    prec_ma = precision_score(y_test_sentimen, y_pred, average='macro')
    recc_mi = recall_score(y_test_sentimen, y_pred, average='micro')
    recc_ma = recall_score(y_test_sentimen, y_pred, average='macro')
    f1_mi = f1_score(y_test_sentimen, y_pred, average='micro')
    f1_ma = f1_score(y_test_sentimen, y_pred, average='macro')

    return  round(acc,4) , round(prec_mi,4), round(recc_mi,4), round(f1_mi,4), round(prec_ma,4), round(recc_ma,4), round(f1_ma,4)

In [ ]:
def plot(test_y, pred_y):
  # labels = unique_labels(label_integritas)
  labels = (-1,0,1)
  # print(labels)
  column = [f'Prediksi {label}' for label in labels]
  indeces = [f'Actual {label}' for label in labels]
  table = pd.DataFrame(confusion_matrix(test_y, pred_y),
                       columns = column, index = indeces)
  return table

In [ ]:
labels = unique_labels(y_test)
labels

array([-1,  0,  1])

## 5.1.Training

In [ ]:
kf_nvb_number = 0
akurasi_nvb_arr = []
prec_nvb_arr_mi = []
prec_nvb_arr_ma = []
rec_nvb_arr_mi = []
rec_nvb_arr_ma = []
f1_nvb_arr_mi = []
f1_nvb_arr_ma = []
score = {
    'Akurasi':[],
    'Presisi_mi':[],
    'Recall_mi':[],
    'F1_mi':[],
    'Presisi_ma':[],
    'Recall_ma':[],
    'F1_ma':[]
}
index_fold = 1

for train_aspek_nvb, test_aspek_nvb in kf.split(X_train, y_train):
  X_train_aspek_nvb, X_test_aspek_nvb = X_train[train_aspek_nvb], X_train[test_aspek_nvb]
  y_train_aspek_nvb, y_test_aspek_nvb = y_train[train_aspek_nvb], y_train[test_aspek_nvb]
 
  kf_nvb_number += 1
 
  # train model
  clf = MultinomialNB()
  clf.fit(X_train_aspek_nvb, y_train_aspek_nvb)
 
  # save model
  # model_nvb = 'vektor_tfidf_sentimen_integritas_nvb.sav'
  pickle.dump(clf, open('/content/drive/My Drive/Thesis/Tesis/program/data/dataset/new_dataset/result_baru/nvb/'+str(nama_sentimen)+'/Fold'+str(index_fold)+'.pkl','wb'))

  # pengujian
  pred_y_nvb = clf.predict(X_test_aspek_nvb.toarray())
  label_asli = y_test_aspek_nvb

# confusion matrix
  acc, prec_mi,prec_ma, recc_mi,recc_ma, f_1_mi,f_1_ma = hitung_score_multiclass(label_asli,pred_y_nvb)

  score['Akurasi'].append(acc)
  score['Presisi_mi'].append(prec_mi)
  score['Presisi_ma'].append(prec_ma)
  score['Recall_mi'].append(recc_mi)
  score['Recall_ma'].append(recc_ma)
  score['F1_mi'].append(f_1_mi)
  score['F1_ma'].append(f_1_ma)

  print(colored("Score fold ke-" + str(index_fold), 'red', attrs=['bold']),'\n',
        'acc training  : ' + str(acc) + "%",'\n',
        'Prec Micro  : '+ str(prec_mi)+ "%", '\n',
        'Prec Macro  : '+ str(prec_ma)+ "%",'\n',
        'recc Micro  : '+ str(recc_mi)+ "%", '\n',
        'recc Macro  : '+ str(recc_ma)+ "%", '\n',
        'f_1 Micro  : '+ str(f_1_mi)+ "%",'\n',
        'f_1 Macro  : '+ str(f_1_ma)+ "%",'\n')
  
  index_fold += 1
from termcolor import colored
print(colored("Rata-rata Akurasi NVB : ", 'red', attrs=['bold']))
print(' acc training: ' + str(np.mean(score['Akurasi'])) + "%", '\n',
      'Prec Micro: ' + str(np.mean(score['Presisi_mi'])) + "%", '\n',
      'Prec Macro: ' + str(np.mean(score['Presisi_ma'])) + "%", '\n',
      'Recall Micro: ' + str(np.mean(score['Recall_mi'])) + "%", '\n',
      'Recall Macro: ' + str(np.mean(score['Recall_ma'])) + "%", '\n',
      'F1 Micro: ' + str(np.mean(score['F1_mi'])) + "%", '\n',    
      'F1 Macro: ' + str(np.mean(score['F1_ma'])) + "%", '\n',
      )
print("Fold-1 : " +str(score['Akurasi'][0]) + "%", 
      "Fold-2 : " + str(score['Akurasi'][1]) + "%")
      # "Fold-3 : " + str(score['Akurasi'][2]) + "%",
      # "Fold-4 : " + str(score['Akurasi'][3]) + "%", 
      # "Fold-5 : " + str(score['Akurasi'][4]) + "%")


--------------------------------------------------
confusion matrix : sentimen_kontinuitas
--------------------------------------------------
           Prediksi -1  Prediksi 0  Prediksi 1
Actual -1          535           0           0
Actual 0             1           0           0
Actual 1             2           0           0 

TP : [535   0   0] TN : [  0 537 536] FP : [3 0 0] FN : [0 1 2] 

Score fold ke-1 
 acc training  : 0.9944% 
 Prec Micro  : 0.9944% 
 Prec Macro  : 0.9944% 
 recc Micro  : 0.9944% 
 recc Macro  : 0.3315% 
 f_1 Micro  : 0.3333% 
 f_1 Macro  : 0.3324% 


--------------------------------------------------
confusion matrix : sentimen_kontinuitas
--------------------------------------------------
           Prediksi -1  Prediksi 0  Prediksi 1
Actual -1          534           0           0
Actual 0             1           0           0
Actual 1             3           0           0 

TP : [534   0   0] TN : [  0 537 535] FP : [4 0 0] FN : [0 1 3] 

Score fold ke-2 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
labels = unique_labels(pred_y_nvb)
labels

array([-1])

## 5.2.Prediksi model

In [ ]:
#Pengujian
indeks=np.where(score["Akurasi"]==np.amax(score['Akurasi']))[0]
model = pickle.load(open('/content/drive/My Drive/Thesis/Tesis/program/data/dataset/new_dataset/result_baru/nvb/'+str(nama_sentimen)+'/Fold'+str(indeks[0]+1)+'.pkl','rb'))
pred_y_nvb = model.predict(X_test)
label_asli = y_test
print("Pengujian " + str(nama_sentimen)," :")
acc, prec_mi,prec_ma, recc_mi,recc_ma, f_1_mi,f_1_ma = hitung_score_multiclass(label_asli,pred_y_nvb)
print(colored('Model terbaik fold ke-'+str(indeks[0]+1), 'red', attrs=['bold']))
print(' Akurasi  : ' + str(acc) + "%",'\n',
      'Presisi Micro  : '+ str(prec_mi)+ "%", '\n',
      'Presisi Macro  : '+ str(prec_ma)+ "%",'\n',
      'Recall Micro  : '+ str(recc_mi)+ "%", '\n',
      'Recall Macro  : '+ str(recc_ma)+ "%", '\n',
      'F1 Micro  : '+ str(f_1_mi)+ "%",'\n',
      'F1 Macro  : '+ str(f_1_ma)+ "%",'\n')

Pengujian sentimen_kontinuitas  :

--------------------------------------------------
confusion matrix : sentimen_kontinuitas
--------------------------------------------------
           Prediksi -1  Prediksi 0  Prediksi 1
Actual -1          267           0           0
Actual 0             1           0           0
Actual 1             1           0           0 

TP : [267   0   0] TN : [  0 268 268] FP : [2 0 0] FN : [0 1 1] 

Model terbaik fold ke-1
 Akurasi  : 0.9926% 
 Presisi Micro  : 0.9926% 
 Presisi Macro  : 0.9926% 
 Recall Micro  : 0.9926% 
 Recall Macro  : 0.3309% 
 F1 Micro  : 0.3333% 
 F1 Macro  : 0.3321% 



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
